<p>Anand Verma</p>
<p>2018imt016</p>
<h3>AIM</h3>
<p>Write a program in C/C++ or python to create a nearest neighbor classifier. The program should take a labeled dataset as input. Use the following dataset in a ratio of 70:30 for training and data.</p>

In [1]:
#Importing necessary libraries
import math
from pprint import pprint
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load the dataset
file_columns = ['sepal_len','sepal_width','petal_len','petal_width','class']
data = pd.read_csv('iris.csv',header=None, names=file_columns)
data.head()

,sepal_len,sepal_width,petal_len,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
#Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)
data['seq'] = data.index
data.head()

,sepal_len,sepal_width,petal_len,petal_width,class,seq
0,6.2,2.2,4.5,1.5,Iris-versicolor,0
1,4.8,3.4,1.6,0.2,Iris-setosa,1
2,7.7,3.8,6.7,2.2,Iris-virginica,2
3,5.0,3.2,1.2,0.2,Iris-setosa,3
4,6.0,2.2,5.0,1.5,Iris-virginica,4


In [4]:
#split dataset into train and test data
dev_size = int(data.shape[0]*0.70)
test_size = int(data.shape[0]*0.30)

# Take first 70% of the data as dev set
dev = data[:dev_size]

# Take last 30% of the data as test set
test = data[test_size:]

In [5]:
# Euclidean distance method
def get_euclidean(row1, row2):
    return math.sqrt(sum([(x1-x2)**2 for x1,x2 in zip(row1,row2)]))

In [6]:
#calculate distance between every point in the dataset and every other point is calculated 
#by using euclidean distance method
dev2 = dev.values
eud = []
l = len(dev2)
for i in range(l):
    eu_distance = []
    for j in range(l):
        if(i!=j):
            index = dev2[j][5]
            ed = get_euclidean(dev2[i][:-2], dev2[j][:-2])
            
            eu_distance.append((ed, index))
    eu_distance.sort(key= lambda x: x[0])
    eu_distance = [i[1] for i in eu_distance]
    eud.append(eu_distance)
dev['euclidean'] = eud

In [7]:
#k closest neighbors are selected for every point in the dev set.
def get_nearest(row, distance_measure,k):
    return row[distance_measure][:k]

In [8]:
#Among the K nearest neighbors, the dominant class is elected and 
#the data point is classified to belong to this class.
def get_dominant_class(df, neighbors):
    classes = df[df['seq'].isin(neighbors)]['class']
    return classes.value_counts().index[0]

In [9]:
#Optimizing K
k = 1
hyper_params = []
acc = {1: {}, 3:{}, 5:{}, 7:{}}

while k <= 7:
    dev['eud_{}'.format(k)] = dev.apply(lambda x: get_nearest(x, 'euclidean',k), axis=1)
    dev['eud_{}_class'.format(k)] = dev['eud_{}'.format(k)].apply(lambda row: get_dominant_class(dev, row))
    hyper_params.append('eud_{}_class'.format(k))
    acc[k]['eud'] = dev[dev['class']==dev['eud_{}_class'.format(k)]].shape[0]/dev.shape[0]
    k+=2

In [10]:
#Observe various hyper parameters
cols = ['class'] + hyper_params
dev[cols].head()

,class,eud_1_class,eud_3_class,eud_5_class,eud_7_class
0,Iris-versicolor,Iris-versicolor,Iris-versicolor,Iris-versicolor,Iris-versicolor
1,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa
2,Iris-virginica,Iris-virginica,Iris-virginica,Iris-virginica,Iris-virginica
3,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa,Iris-setosa
4,Iris-virginica,Iris-versicolor,Iris-versicolor,Iris-virginica,Iris-virginica


In [11]:
pprint(acc)

{1: {'eud': 0.9619047619047619},
 3: {'eud': 0.9619047619047619},
 5: {'eud': 0.9714285714285714},
 7: {'eud': 0.9714285714285714}}


In [12]:
#Calculate euclidean distance between every point in the test dataset 
#with every point in the dev set and pick 3 nearest neighbors
test['seq'] = test.index
test2 = test.values
test_eud = []
l = len(test)
for i in range(l):
    test_eu_distance = []
    for j in range(len(dev)):
        index = dev2[j][5]
        ed = get_euclidean(test2[i][:-2], dev2[j][:-2])
        test_eu_distance.append((ed, index))
        
    test_eu_distance.sort(key= lambda x: x[0])
    test_eu_distance = [i[1] for i in test_eu_distance]
    test_eud.append(test_eu_distance)
    
test['euclidean'] = test_eud

In [13]:
#Since we obtained 3 as optimal value for the hyperparameter k, 
#we find 3 nearest neighbors between every point in the test set
# and every point in the dev set using Euclidean Distance
test['eu'] = test.apply(lambda x: get_nearest(x, 'euclidean',3), axis=1)
test[file_columns+['eu']].head()

,sepal_len,sepal_width,petal_len,petal_width,class,eu
45,5.4,3.9,1.3,0.4,Iris-setosa,"[45, 16, 92]"
46,6.3,3.4,5.6,2.4,Iris-virginica,"[46, 87, 90]"
47,7.7,3.0,6.1,2.3,Iris-virginica,"[47, 25, 71]"
48,6.3,3.3,4.7,1.6,Iris-versicolor,"[48, 9, 57]"
49,5.1,3.3,1.7,0.5,Iris-setosa,"[49, 43, 63]"


In [14]:
#Pick the most dominant class among 3 nearest neighbors
test['eu_class'] = test['eu'].apply(lambda row: get_dominant_class(dev, row))
test[file_columns+['eu_class']].head()

,sepal_len,sepal_width,petal_len,petal_width,class,eu_class
45,5.4,3.9,1.3,0.4,Iris-setosa,Iris-setosa
46,6.3,3.4,5.6,2.4,Iris-virginica,Iris-virginica
47,7.7,3.0,6.1,2.3,Iris-virginica,Iris-virginica
48,6.3,3.3,4.7,1.6,Iris-versicolor,Iris-versicolor
49,5.1,3.3,1.7,0.5,Iris-setosa,Iris-setosa


In [15]:
y_test=test["class"].tolist()

In [16]:
y_pred=test["eu_class"].tolist()

In [17]:
# Accuracy, Confusion Matrix, Classification report of KNN classifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
print(confusion_matrix(y_test, y_pred))
print("Accuracy:",accuracy_score(y_test,y_pred)*100)
print(classification_report(y_test, y_pred))


[[32  0  0]
 [ 0 38  3]
 [ 0  1 31]]
Accuracy: 96.19047619047619
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        32
Iris-versicolor       0.97      0.93      0.95        41
 Iris-virginica       0.91      0.97      0.94        32

       accuracy                           0.96       105
      macro avg       0.96      0.97      0.96       105
   weighted avg       0.96      0.96      0.96       105

